# Evaluation of RAG architectures

In [1]:
from llms.rag.faiss import FAISS
from llms.rag.coala import CoALA
from llms.agents.react import ReActAgent
from llms.clients.gpt import GPTClient
from llms.settings import settings
from llms.rag.tools import Tools
from llms.rag.faiss import DistanceMetric
from llms.evaluation.code import evaluate_code_generation, ConfigGrid, RAG, RAGRetriever

/opt/homebrew/Caskroom/miniforge/base/envs/llms-using-rag-for-docs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpt_4_client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id='gpt-4-32k',
    max_response_tokens=100,
    temperature=0.0,
)

gpt_35_client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id='gpt-35-turbo-16k',
    max_response_tokens=100,
    temperature=0.0,
)

In [3]:
print(settings.STANDARD_SYSTEM_INSTRUCTION)


    You are an AI assistant who can write code using pandas.
    All necessary code that is part of the answer must be in a single python function called response_function.
    If you have one argument given to you in the user prompt, write your response function so that it takes one argument.
    If you have two arguments given in the user prompt, write your response function so that it takes two arguments.
    Do not write any text or code outside this function when constructing an answer or action.
    In the first line of code inside the function please always import pandas as pd or pyarrow as pa, depending on what you need.
    Pandas, numpy and pyarrow are the only non-standard packages you are allowed to use.

    Always use the following JSON response format:
    {
        "Question": the input question you must answer
        "Thought": you should always think about what to do
        "Action": "def response_function(arguments as required by the user prompt):
code goes here"


In [10]:
config = ConfigGrid(
    llms=[gpt_4_client, gpt_35_client],
    rag=RAG(
        retrievers=[RAGRetriever.CoALA],
        distance_metrics=[DistanceMetric.EUCLIDEAN_DISTANCE],
        num_search_results=[3],
        similarity_search_score_thresholds=[0.0],
        texts=["Hello", "test123"],
        text_chunk_sizes=[128],
    )
)

evaluate_code_generation(config, [])

0it [00:00, ?it/s]/4 [00:00<?, ?it/s]
13409 - DEBUG - Starting new HTTPS connection (1): business-decision-simulator-experiments-qpti6y8s.authentication.eu12.hana.ondemand.com:443
13409 - DEBUG - https://business-decision-simulator-experiments-qpti6y8s.authentication.eu12.hana.ondemand.com:443 "POST /oauth/token?grant_type=client_credentials HTTP/1.1" 200 None
13409 - DEBUG - Starting new HTTPS connection (1): azure-openai-serv-i057149.cfapps.eu12.hana.ondemand.com:443
13409 - DEBUG - https://azure-openai-serv-i057149.cfapps.eu12.hana.ondemand.com:443 "POST /api/v1/embeddings HTTP/1.1" 200 19629
13409 - DEBUG - Starting new HTTPS connection (1): business-decision-simulator-experiments-qpti6y8s.authentication.eu12.hana.ondemand.com:443
13409 - DEBUG - https://business-decision-simulator-experiments-qpti6y8s.authentication.eu12.hana.ondemand.com:443 "POST /oauth/token?grant_type=client_credentials HTTP/1.1" 200 None
13409 - DEBUG - Starting new HTTPS connection (1): azure-openai-serv-i05